# Scraping Turtles Website Content

We need to understand the process of how the process is happend behined the seens

then we need to install the following:
1. requests (This will used to communicate with the brower to get the response)
2. beautifulsoup4 (this will convert unredable data which came from the requests and make it more structured to deal with by using parser)

In [440]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [441]:
base_url = 'https://www.scrapethissite.com'
card_url = "https://www.scrapethissite.com/pages/frames/?frame=i"
inside_card_url = 'https://www.scrapethissite.com/pages/frames/?frame=i&family=Carettochelyidae'
current_link = base_url

r = requests.get(base_url) # will send a req for get and will save the res into a variable called 'r'
print(r)
print(f'Without BeautifulSoup: {r.content}') # This will print unreadable data and we cant deal with it.

# Here the power of BeautifulSoup comes to convert the 'r.content'to readable data.
soup = BeautifulSoup(r.content, 'html') # here what happend is we called a function 'BeautifulSoup' to parse the unreadable 'r.content' to the readable and specify what we want to parse in our setuation is html
print(soup)

# Now we can identify the element what we want to fetch by their classes, id's, elements

<Response [200]>
Without BeautifulSoup: b'<!doctype html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n    <title>Scrape This Site | A public sandbox for learning web scraping</title>\n    <link rel="icon" type="image/png" href="/static/images/scraper-icon.png" />\n\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta name="description" content="A public sandbox for learning web scraping">\n\n    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" rel="stylesheet" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" crossorigin="anonymous">\n    <link href=\'https://fonts.googleapis.com/css?family=Lato:400,700\' rel=\'stylesheet\' type=\'text/css\'>\n    <link rel="stylesheet" type="text/css" href="/static/css/styles.css">\n\n    \n\n  </head>\n\n  <body>\n    <nav id="site-nav">\n            <div class="

In [442]:
r = requests.get(card_url)
soup = BeautifulSoup(r.content, 'html')
cards = soup.find_all('div', class_ = "col-md-4 turtle-family-card")
all_cards = {'turtle_name': [], 'turtle_family_name':[], 'discovered_year' : [], 'discovered_by' : []}
for card in cards:
    turtle_tag = card.find('h3', class_ = 'family-name')
    if turtle_tag:
        turtle_name = turtle_tag.text.strip()
        all_cards['turtle_name'].append(turtle_name)

    link_tag = card.find('a', href = True)
    if not  link_tag:
        all_cards['turtle_family_name'].append("no link")
        continue

    turtle_details_url = base_url + link_tag['href']
    details_url = requests.get(turtle_details_url)
    detail_soup = BeautifulSoup(details_url.content, 'html.parser')

    details_p = detail_soup.find('p', class_ = "lead")
    known_as = details_p.find('strong', class_ = 'common-name')
    if known_as:
        turtle_known_as = known_as.text.strip()
        all_cards['turtle_family_name'].append(turtle_known_as)

    else:
        all_cards['turtle_family_name'].append("no known as")
        continue
    
    year = r"\b(1[5-9][0-9]{2}|20[0-2][0-9]|2030)\b"

    details_p = detail_soup.find_all('p', class_ = "lead")

    if details_p:
        for p in details_p:
            paragraph_text = p.get_text(strip=True)
            matches = re.findall(year, paragraph_text)
            if matches:
                all_cards['discovered_year'].append(matches[0])


    discovered_by = r"\bby\s+([A-Z][a-zA-Z\-]*)"
    for p in details_p:
        paragraph_text_by = p.get_text(strip=True)
        matches = re.findall(discovered_by, paragraph_text_by)
        if matches:
            all_cards['discovered_by'].append(matches[0])


print(all_cards)


{'turtle_name': ['Carettochelyidae', 'Cheloniidae', 'Chelydridae', 'Dermatemydidae', 'Dermochelyidae', 'Emydidae', 'Geoemydidae', 'Kinosternidae', 'Platysternidae', 'Testudinidae', 'Trionychidae', 'Chelidae', 'Pelomedusidae', 'Podocnemididae'], 'turtle_family_name': ['Pig-nosed turtle', 'Sea turtles', 'Snapping turtles', 'Central American river turtle', 'Leatherback sea turtle', 'Pond or water turtles', 'Asian river, leaf, roofed or Asian box turtles', 'Mud or musk turtles', 'Big-headed turtle', 'Tortoises', 'Softshell turtles', 'Austro-American sideneck turtles', 'Afro-American sideneck turtles', 'Madagascar big-headed, Big-headed Amazon River turtle and South American sideneck river turtles'], 'discovered_year': ['1887', '1811', '1831', '1870', '1843', '1815', '1868', '1857', '1869', '1788', '1826', '1831', '1868', '1869'], 'discovered_by': ['Boulenger', 'Oppel', 'Gray', 'Gray', 'Fitzinger', 'Rafinesque', 'Theobald', 'Agassiz', 'Gray', 'Batsch', 'Fitzinger', 'Gray', 'Cope', 'Gray']}


In [443]:
df = pd.DataFrame(all_cards)
df

,turtle_name,turtle_family_name,discovered_year,discovered_by
0,Carettochelyidae,Pig-nosed turtle,1887,Boulenger
1,Cheloniidae,Sea turtles,1811,Oppel
2,Chelydridae,Snapping turtles,1831,Gray
3,Dermatemydidae,Central American river turtle,1870,Gray
4,Dermochelyidae,Leatherback sea turtle,1843,Fitzinger
5,Emydidae,Pond or water turtles,1815,Rafinesque
6,Geoemydidae,"Asian river, leaf, roofed or Asian box turtles",1868,Theobald
7,Kinosternidae,Mud or musk turtles,1857,Agassiz
8,Platysternidae,Big-headed turtle,1869,Gray
9,Testudinidae,Tortoises,1788,Batsch
